# MusicAI
## Sztuczna Inteligencja - projekt 
### Część 3 - prosta sieć neuronowa
Autorzy: Jakub Ochnik, Adam Karabiniewicz, Marcel Bieniek
___


Importing necessary libraries and packages

In [ ]:
import numpy as np
import math
import json
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import matplotlib.pyplot as plt

Constants

In [ ]:
DATA_PATH = "Data\\data_full.json"

Loading dataset

In [ ]:
def load_data(data_path):
    """Loads training dataset from json file.
        :param data_path (str): Path to json file containing data
        :return X (ndarray): Inputs
        :return y (ndarray): Targets
    """

    with open(data_path, "r") as fp:
        data = json.load(fp)

    # convert lists to numpy arrays
    """
    inputs -> X
    outputs -> y
    """

    X = np.array(data["mfcc"])
    y = np.array(data["labels"])

    print("Data succesfully loaded!")

    return  X, y

In [ ]:
X, y = load_data(DATA_PATH)

print(len(X))
print(len(y))

Splitting the data into train and test sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

In [ ]:
print(X.shape)

Build the network

In [ ]:
# input layer
# 3 hidden layers
# simple multilayered perceptron (5)
# fully connected (dense) layers
model = keras.Sequential([
    # input layer
    # flatten takes multidim array and flattens it out
    # json contains 3D array, index 0 represents different segments (different segments), 1 & 2 represent actual data shape
    keras.layers.Flatten(input_shape=(X.shape[1],X.shape[2])),
    # MOZNA FAJNIE OPISAC RELU W SPRAWKU
    # ReLU is better for training than sigmoid, better convergence, reduced likelihood of vanishing gradient
    # 1st hidden layer
    keras.layers.Dense(512, activation="relu", kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Dropout(0.3),
    # 2nd hidden layer
    keras.layers.Dense(256, activation="relu", kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Dropout(0.3),
    # 3rd hidden layer
    keras.layers.Dense(64, activation="relu", kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Dropout(0.3),
    # output layer
    # 10 neurons = 10 genres
    # softmax
    keras.layers.Dense(10, activation="softmax")    
])

Compiling the network

In [ ]:
# Adam - extension of classic gradient descent, effective
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.summary()

Training the network

In [ ]:
# batching - the way of how we train network
# stochastic gradient descent (quick, inaccurate), full batch (compute gradient on the whole training set, slow, accurate), mini-batch (compute gradient on a subset of dataset, 16-128 samples, the best of 2 worlds)
# batch_size - number of samples of mini-batch
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32)

Plot accuracy and error over epochs

In [ ]:
def plot_history(history):
    fig, axs = plt.subplots(2)
    # accuracy subplot
    axs[0].plot(history.history["accuracy"], label="train accuracy")
    axs[0].plot(history.history["val_accuracy"], label="test accuracy")

    axs[0].set_ylabel("Accuracy")
    axs[0].legend(loc="lower right")
    axs[0].set_title("Accuracy eval")

    # error subplot
    axs[1].plot(history.history["loss"], label="train error")
    axs[1].plot(history.history["val_loss"], label="test error")

    axs[1].set_ylabel("Error")
    axs[1].set_xlabel("Epoch")
    axs[1].legend(loc="upper right")
    axs[1].set_title("Error eval")

    plt.show()

In [ ]:
plot_history(history)

In [ ]:
test_error, test_accuracy = model.evaluate(X_test, y_test, verbose = 1)
print("Model accuracy: {}".format(test_accuracy))